In [ ]:
println("Hello World")

In [ ]:
using Pkg
Pkg.add(PackageSpec(name="NextGenSeqUtils", rev="1.0", url = "https://github.com/MurrellGroup/NextGenSeqUtils.jl.git"))

In [ ]:
Pkg.add.(["PyPlot", "StatsBase", "Compose", "GraphPlot", "LightGraphs", "Colors"])

In [ ]:
Pkg.add(url="https://github.com/MurrellGroup/NextGenSeqUtils.jl")
Pkg.add(url="https://github.com/MurrellGroup/DPMeansClustering.jl")
Pkg.add(url="https://github.com/MurrellGroup/RobustAmpliconDenoising.jl")

In [ ]:
using NextGenSeqUtils,PyPlot,GraphPlot,LightGraphs,Colors,Compose

In [ ]:
import Pkg
Pkg.add("BioSequences")

## Denoising based on https://nextjournal.com/Murrell-Lab/scfv-fad-analysis

In [ ]:
all_pan4 = "all_pan4.fastq"
length_vs_qual(all_pan4, alpha = 0.02)

In [ ]:
jamy_2020 = "/home/lubo/code/wRajter/vampyrella_2023/raw_data/PacBio/Jamy_2019_rDNA/cell/raw/ERR2355431.fastq"
length_vs_qual(jamy_2020, alpha = 0.02)

In [ ]:
#Filter and export filtered data
fastq_filter(jamy_2020, "jamy_filt.fastq", error_rate = 0.01, min_length = 3000, max_length = 6000);

In [ ]:
seqs,phreds,seq_names = read_fastq("jamy_filt.fastq");

In [ ]:
#Use the demux_dict() function to orient the sequences, and discard those whose primers do not match what is expected
fwd_primers = ["GGCAAGTCTGGTGCCAG"]
rev_primers = ["GACGAGGCATTTGGCTACCTT"]
demux_dic = demux_dict(seqs,fwd_primers,rev_primers,verbose=true,phreds = phreds,tol_one_error = true);

In [ ]:
#Trim off the full primer sequences (absolutely essential when primers contain degenerate/ambig bases)
trimmed = [double_primer_trim(j[1],j[2],"GGCAAGTCTGGTGCCAG","GACGAGGCATTTGGCTACCTT") for j in demux_dic[(1,1)]]
reads,phredarr = [s[1] for s in trimmed],[s[2] for s in trimmed]

In [ ]:
#Run the Fast Amplicon Denoising algorithm
@time centroids,sizes = FAD(reads,phreds = phreds);

In [ ]:
#Sort sequences by size
size_sortperm = reverse(sortperm(sizes))
centroids = centroids[size_sortperm]
sizes = sizes[size_sortperm]

In [ ]:
#Export size-sorted FAD reconstructed variants
write_fasta("jamy_2020_FAD.fasta",centroids,names = ["seq_$(j)_$(sizes[j])" for j in 1:length(sizes)])

## Denoising based on https://github.com/MurrellGroup/RobustAmpliconDenoising.jl

In [2]:
using Pkg;
Pkg.add(url="https://github.com/MurrellGroup/NextGenSeqUtils.jl");
Pkg.add(url="https://github.com/MurrellGroup/DPMeansClustering.jl");
Pkg.add(url="https://github.com/MurrellGroup/RobustAmpliconDenoising.jl");

    Updating git-repo `https://github.com/MurrellGroup/NextGenSeqUtils.jl`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
    Updating git-repo `https://github.com/MurrellGroup/DPMeansClustering.jl`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
    Updating git-repo `https://github.com/MurrellGroup/RobustAmpliconDenoising.jl`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [3]:
using NextGenSeqUtils, RobustAmpliconDenoising

In [4]:
jamy_2020_filt = "/home/lubo/code/wRajter/vampyrella_2023/raw_data/PacBio/Jamy_2019_rDNA/cell/filtered/ERR2355431.fastq"

seqs, QVs, seq_names = read_fastq(jamy_2020_filt)

(["..."], Vector{Int8}[[93, 93, 93, 93, 93, 93, 93, 93, 93, 93  …  93, 93, 93, 93, 93, 93, 93, 93, 93, 71], [93, 83, 93, 93, 59, 88, 90, 93, 69, 88  …  93, 93, 93, 90, 88, 68, 49, 64, 93, 56], [93, 93, 77, 93, 93, 93, 93, 77, 93, 93  …  93, 93, 93, 93, 93, 93, 93, 93, 93, 93], [65, 93, 93, 93, 65, 93, 93, 93, 77, 93  …  90, 93, 93, 93, 86, 93, 76, 93, 93, 89], [93, 93, 93, 93, 42, 93, 93, 93, 93, 93  …  93, 93, 93, 93, 93, 93, 93, 93, 93, 93], [93, 93, 93, 93, 93, 93, 93, 79, 93, 93  …  93, 93, 93, 93, 93, 93, 93, 93, 93, 93], [93, 93, 93, 72, 93, 93, 93, 93, 93, 93  …  93, 93, 93, 93, 93, 93, 93, 93, 93, 93], [93, 93, 93, 93, 93, 93, 93, 93, 93, 93  …  93, 93, 93, 93, 93, 93, 93, 93, 93, 93], [93, 93, 93, 93, 93, 93, 93, 93, 93, 93  …  93, 93, 93, 86, 93, 90, 93, 93, 93, 93], [88, 85, 88, 90, 67, 71, 64, 58, 41, 83  …  75, 89, 93, 93, 93, 90, 64, 65, 88, 52]  …  [70, 72, 93, 93, 53, 85, 93, 93, 73, 87  …  93, 93, 93, 93, 93, 72, 93, 93, 93, 33], [93, 93, 36, 93, 93, 93, 93, 93, 93, 93

In [5]:
templates,template_sizes,template_indices = denoise(seqs)

Your inputs have begun rough clustering...
cycle 1. n_clusters=209
cycle 2. n_clusters=209
cycle 3. n_clusters=209
Converged!
n_keep=209 after removing empty clusters.
Your inputs have begun fine clustering...
Splitting cluster number 1
Splitting cluster number 2
Splitting cluster number 3
Splitting cluster number 4
Splitting cluster number 5
Splitting cluster number 6
Splitting cluster number 7
Splitting cluster number 8
Splitting cluster number 9
Splitting cluster number 10
Splitting cluster number 11
Splitting cluster number 12
Splitting cluster number 13
Splitting cluster number 14
Splitting cluster number 15
Splitting cluster number 16
Splitting cluster number 17
Splitting cluster number 18
Splitting cluster number 19
Splitting cluster number 20
Splitting cluster number 21
Splitting cluster number 22
Returning 34 templates
Finding consensus sequences...
Reassigning singleton inputs to consensus sequences to improve accuracy of frequencies
After reassigning, there are 33 templates


(["..."], [127, 96, 39, 42, 23, 37, 30, 59, 24, 22  …  39, 13, 16, 43, 22, 6, 17, 12, 12, 10], Vector{Any}[[5, 11, 21, 32, 49, 78, 87, 103, 110, 120  …  841, 846, 854, 881, 900, 915, 916, 922, 925, 927], [19, 26, 40, 43, 57, 75, 96, 101, 109, 122  …  813, 822, 827, 844, 852, 857, 893, 899, 904, 920], [29, 53, 59, 81, 91, 117, 123, 160, 166, 168  …  623, 658, 732, 735, 752, 801, 838, 840, 928, 935], [42, 79, 99, 112, 150, 152, 189, 191, 200, 202  …  741, 757, 772, 778, 810, 842, 856, 863, 894, 907], [41, 145, 197, 312, 320, 327, 330, 383, 412, 415  …  554, 574, 688, 819, 825, 829, 853, 879, 884, 934], [16, 23, 55, 64, 84, 105, 116, 132, 134, 156  …  624, 634, 660, 704, 770, 776, 847, 859, 914, 939], [9, 17, 24, 90, 93, 98, 118, 164, 165, 179  …  627, 655, 699, 706, 746, 862, 870, 882, 912, 917], [33, 45, 61, 62, 72, 86, 102, 111, 114, 146  …  786, 795, 799, 832, 851, 875, 889, 891, 923, 929], [20, 37, 85, 180, 186, 188, 219, 230, 240, 451  …  602, 663, 700, 703, 769, 771, 779, 831, 839,

In [6]:
write_fasta("jamy_2020_denoised.fasta",templates,names = ["seq$(j)_$(template_sizes[j])" for j in 1:length(template_sizes)])